# Novelty analysis
In this notebook, we'll look at the trade-offs between novelty and accuracy in semantic matching.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('../../')

In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)

import takco
import pandas as pd

conf = takco.Config('resources/config-dbpedia.toml')
tables = takco.TableSet._load('output/t2d-v2-baseline/1-link/*')
scored_tables = tables.score('t2d-v2', keycol_only=True, **conf)

takco.preview( t for t in scored_tables if any(t.get('gold', {}).values()) )

INFO:root:Loading data from resources/t2d_fix.csv
INFO:root:Loaded 514 annotated tables


In [3]:
report = scored_tables.make_report(keycol_only=True)
scores = report['scores']
pd.DataFrame.from_dict(scores, orient='index')

INFO:root:Collected 26106 gold and 21222 pred for task entities
INFO:root:Collected 671 gold and 388 pred for task properties
INFO:root:Collected 237 gold and 244 pred for task classes


,precision,recall,f1-score,support,predictions
entities,0.802563,0.652417,0.719743,26106,21222
properties,0.384021,0.222057,0.281398,671,388
classes,0.864754,0.890295,0.877339,237,244


In [4]:
novelty_tables = scored_tables.triples().novelty('dbpedia_t2ksubset', **conf)
len(list(novelty_tables))

514

In [5]:
report = novelty_tables.report(keycol_only=True)
report

INFO:root:Collected 26106 gold and 21222 pred for task entities
INFO:root:Collected 671 gold and 388 pred for task properties
INFO:root:Collected 237 gold and 244 pred for task classes


{'scores': {'entities': {'precision': 0.8025633776269908,
   'recall': 0.652417068873056,
   'f1-score': 0.7197430696416497,
   'support': 26106,
   'predictions': 21222},
  'properties': {'precision': 0.38402061855670105,
   'recall': 0.22205663189269748,
   'f1-score': 0.2813975448536355,
   'support': 671,
   'predictions': 388},
  'classes': {'precision': 0.8647540983606558,
   'recall': 0.890295358649789,
   'f1-score': 0.8773388773388774,
   'support': 237,
   'predictions': 244}},
 'novelty': {'dbpedia_t2ksubset': {'label': {'tp_existing': 9219,
    'fn_existing': 3690,
    'fp_existing': 7715,
    'tp_attnovel': 211,
    'fn_attnovel': 2061,
    'fp_attnovel': 1638,
    'tp_valnovel': 26,
    'fn_valnovel': 1083,
    'fp_valnovel': 2266,
    'attnovel_precision': 0.1141157382368848,
    'attnovel_recall': 0.09286971830985916,
    'attnovel_f1': 0.10240232953166709,
    'valnovel_precision': 0.011343804537521814,
    'valnovel_recall': 0.023444544634806132,
    'valnovel_f1': 0.